In [1]:
import sys
sys.path.append("../base")

import boilerplate
from pyrtl import *

In [12]:
reset_working_block()

pc = Register(name='pc', bitwidth=4)
sp = Register(name='sp', bitwidth=4)
mem = MemBlock(name='mem', bitwidth=4, addrwidth=4)
rom = RomBlock(name='rom', bitwidth=5, addrwidth=4, romdata=[0x15, 0x18, 0, 0xf],
               pad_with_zeros=True)  # needed for C compilation

out = Output(name='out', bitwidth=4)

In [13]:
instr = rom[pc]
out <<= mem[sp]

with conditional_assignment:
    with instr[4] == 1:   # PUSH
        mem[sp] |= instr[0:4]
        sp.next |= sp + 1
        pc.next |= pc + 1
    with instr == 0:      # POP
        sp.next |= sp - 1
        pc.next |= pc + 1

In [14]:
# Debug ports -- so that we see something during simulation
d_sp = Output(name='d_sp', bitwidth=4)
d_pc = Output(name='d_pc', bitwidth=4)

d_sp <<= sp
d_pc <<= pc

### Simulation in C

In [15]:
import os
from simulate import CCompiledSimulation

In [16]:
CCompiledSimulation(out_dir="obj")

In [19]:
os.system("make")

clang++ -O3 -Iobj -std=c++11 -DNO_DEBUG_MEM -c -o obj/csim_main.o simulate/csim_main.cpp
clang++ -o bin/csim obj/csim.o obj/csim_main.o


0

In [25]:
os.environ['DEBUG_CPU'] = '1'
os.environ['MAX_CYCLES'] = '20'
os.system('bin/csim')

0000 0000 
0001 0001 
0002 0002 
0003 0001 
0003 0001 
0003 0001 
0003 0001 
0003 0001 
0003 0001 
0003 0001 
0003 0001 
0003 0001 
0003 0001 
0003 0001 
0003 0001 
0003 0001 
0003 0001 
0003 0001 
0003 0001 
0003 0001 
[info] Simulation time: 0.086125ms
[info] # cycles: 20
[info] 0.232221MHz


0